In [1]:
from monitor.monitor import Monitor
from monitor.lsa import LSA
from monitor.mdsa import MDSA

EXTRACT_NEW_TRACES_AND_PREDICTIONS = False
METRIC_TYPES = [ LSA, MDSA ]

In [2]:
monitor = Monitor()
print(monitor)

LUNAR LANDER MONITOR PROPERTIES:

	 Q-func Model Path: 		./output/lunar_lander.keras
	 Q-Model Num Actions: 		4

	 Training Data Path: 		./output/latest_buffer_states.npy
	 Output Video Path: 		./output/lunar_lander.mp4
	 Output Video FPS: 			30

	 Monitor Trace Path: 		./output/latest_traces_{}.npy
	 Model Prediction Path: 	./output/latest_preds.npy


In [3]:
if EXTRACT_NEW_TRACES_AND_PREDICTIONS:
    monitor.extract_traces_and_preds()
else:
    print("Extracting new traces and predictions from scratch was not requested.")
# end if-else block

Extracting new traces and predictions from scratch was not requested.


In [ ]:
print("TODO - generate some bogus data to use as test inputs")

In [4]:
monitor.calculate_surprise_scores(METRIC_TYPES)

<class 'monitor.lsa.LSA'>
<class 'monitor.mdsa.MDSA'>
